In [ ]:
import torch
import torchvision

import matplotlib.pyplot as plt
import torch
import torchvision

import requests

from torch import nn
from torchvision import transforms

from pathlib import Path

try:
  from torchinfo import summary
except:
  print("[INFO] Couldn't find torchinfo... installing it.")
  !pip install -q torchinfo
  from torchinfo import summary


try:
  from modular import download_image_data, data_setup, train_model, save
except:
  # Get the going_modular scripts
  print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
  !git clone https://github.com/AaronJ59/Projects.git
  !mv Projects/BiteID/modular/modular modular
  !mv Projects/BiteID/models models
  from modular import download_image_data, data_setup, train_model, save
  !rm -rf Projects


[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.
Cloning into 'Projects'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 159 (delta 30), reused 10 (delta 1), pack-reused 54 (from 2)
Receiving objects: 100% (159/159), 109.85 MiB | 27.42 MiB/s, done.
Resolving deltas: 100% (39/39), done.
mv: cannot stat 'Projects/BiteID/models': No such file or directory


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# I must manually import the effnetb2 model state dict and the large dataset zip file

In [ ]:
pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import os
import zipfile

### Upload multiclass dataset

In [ ]:
from zipfile import ZipFile

directory_name = "data/large_pizza_hamburger_lasagna_sushi_steak"

try:
  os.mkdir("data")
  os.mkdir(directory_name)
  print(f"Directory {directory_name} created")
  with zipfile.ZipFile("large_pizza_hamburger_lasagna_sushi_steak.zip", "r") as zip_ref:
    print("Writing zip file to destination")
    zip_ref.extractall(directory_name)
except FileExistsError:
  print(f"Directory already exists")

Directory data/large_pizza_hamburger_lasagna_sushi_steak created
Writing zip file to destination


In [ ]:
train_dir = "data/large_pizza_hamburger_lasagna_sushi_steak/train"
test_dir = "data/large_pizza_hamburger_lasagna_sushi_steak/test"

In [ ]:
def walk_through_dir(dir_path):
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


walk_through_dir("data/")

There are 1 directories and 0 images in 'data/'.
There are 2 directories and 0 images in 'data/large_pizza_hamburger_lasagna_sushi_steak'.
There are 5 directories and 0 images in 'data/large_pizza_hamburger_lasagna_sushi_steak/train'.
There are 0 directories and 523 images in 'data/large_pizza_hamburger_lasagna_sushi_steak/train/hamburger'.
There are 0 directories and 526 images in 'data/large_pizza_hamburger_lasagna_sushi_steak/train/pizza'.
There are 0 directories and 541 images in 'data/large_pizza_hamburger_lasagna_sushi_steak/train/steak'.
There are 0 directories and 489 images in 'data/large_pizza_hamburger_lasagna_sushi_steak/train/sushi'.
There are 0 directories and 546 images in 'data/large_pizza_hamburger_lasagna_sushi_steak/train/lasagna'.
There are 5 directories and 0 images in 'data/large_pizza_hamburger_lasagna_sushi_steak/test'.
There are 0 directories and 197 images in 'data/large_pizza_hamburger_lasagna_sushi_steak/test/hamburger'.
There are 0 directories and 194 image

### Upload binary dataset (if testing/experimentation needed)

In [ ]:
import zipfile
with zipfile.ZipFile("binary_dataset.zip", "r") as zip_ref:
  zip_ref.extractall()

In [ ]:
import gradio as gr
import torch

In [ ]:
def create_effnetb2_model(num_classes: int,
                          seed:int=42):


  # Import efficientnetb2

  # Get the weights, transforms of the model and make the model
  efficientnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  effnetb2_transform = efficientnetb2_weights.transforms()
  effnetb2 = torchvision.models.efficientnet_b2(weights=efficientnetb2_weights)

  # Freeze the base layers of efficientnetb2
  for param in effnetb2.parameters():
    param.requires_grad = False



  # Change classifier head of effnetb2 to suit 5 classes
  torch.manual_seed(seed)
  effnetb2.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408, out_features=num_classes),
  )

  return effnetb2, effnetb2_transform




effnetb2, effnetb2_transform = create_effnetb2_model(num_classes=5,
                                                           seed=42)

# Create effnetb2 model for binary classifier
effnetb2_binary, effnetb2_transform = create_effnetb2_model(num_classes=1,
                                                            seed=42)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 70.2MB/s]


In [ ]:
# Load state dict for multiclass model

effnetb2.load_state_dict(torch.load(f="large_dataset_effnetb2_90_percent_accuracy_10_epochs.pth", map_location=torch.device("cpu")))

<ipython-input-10-b80b77f5f5d5>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  effnetb2.load_state_dict(torch.load(f="large_dataset_effnetb2_90_percent_accuracy_10_epochs.

<All keys matched successfully>

In [ ]:
# Load state dict for binary model

effnetb2_binary.load_state_dict(torch.load(f="effnetb2_99_percent_accuracy_binary_classification.pth", map_location=torch.device("cpu")))

<ipython-input-11-c9bcb0b1e3eb>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  effnetb2_binary.load_state_dict(torch.load(f="effnetb2_99_percent_accuracy_binary_classifica

<All keys matched successfully>

In [ ]:
image_paths = list(Path(test_dir).glob("*/*.jpg"))

In [ ]:
class_names = ['hamburger', 'lasagna', 'pizza', 'steak', 'sushi']

In [ ]:
import pathlib
import torch

from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import List, Dict

def pred_and_store(image_paths: List[pathlib.Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   device: str = "cuda" if torch.cuda.is_available() else "cpu") -> List[Dict]:

  pred_list = []

  model.to(device)

  for path in tqdm(image_paths):

    pred_dict = {}

    class_name = path.parent.stem
    pred_dict["class"] = class_name

    pred_dict["path"] = path

    start_time = timer()

    image = Image.open(path)

    # Transform the image, add batch dimension and put image on target device
    image_transformed = transform(image).unsqueeze(dim=0).to(device)

    model.eval()

    with torch.inference_mode():
      pred_logits = model(image_transformed)

      end_time = timer()


      pred_probs = torch.softmax(pred_logits, dim=1)
      pred_label = torch.argmax(pred_probs, dim=1)

      pred_dict["pred_prob"] = round(pred_probs.max().cpu().item(), 4)
      pred_dict["pred_class"] = class_names[pred_label.cpu()]



    pred_dict["pred_time"] = round(end_time - start_time, 4)
    pred_dict["correct_pred"] = class_name == class_names[pred_label.cpu()]

    # Append created dictionary to the list
    pred_list.append(pred_dict)

  return pred_list




In [ ]:
effnetb2_pred_list = pred_and_store(image_paths=image_paths,
                                   model=effnetb2,
                                   transform=effnetb2_transform)

  0%|          | 0/988 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
effnetb2_pred_df = pd.DataFrame(effnetb2_pred_list)

effnetb2_pred_df.head()

,class,path,pred_prob,pred_class,pred_time,correct_pred
0,hamburger,data/large_pizza_hamburger_lasagna_sushi_steak...,0.9130,hamburger,0.0380,True
1,hamburger,data/large_pizza_hamburger_lasagna_sushi_steak...,0.9996,hamburger,0.0399,True
2,hamburger,data/large_pizza_hamburger_lasagna_sushi_steak...,0.9946,hamburger,0.0349,True
3,hamburger,data/large_pizza_hamburger_lasagna_sushi_steak...,0.9946,hamburger,0.0393,True
4,hamburger,data/large_pizza_hamburger_lasagna_sushi_steak...,0.6806,hamburger,0.0360,True


In [ ]:
effnetb2_pred_df.correct_pred.value_counts()

,count
correct_pred,
True,893
False,95


Sucessfully loaded in the state dict

## Implement gradio interface

In [ ]:
binary_class_names = ["0_non-food", "1_food"]

In [ ]:
import time

effnetb2_binary.to(device)
effnetb2.to(device)


def image_classification(image):



  start = time.time()

  # First, pass image through binary classifier to see if the image is food
  image_transformed = effnetb2_transform(image).unsqueeze(dim=0).to(device)

  effnetb2_binary.eval()

  with torch.inference_mode():
    pred_logit = effnetb2_binary(image_transformed)

    pred_prob = torch.sigmoid(pred_logit)
    pred_label = (pred_prob >= 0.5).long()
    print(pred_label)
    # if the image is food:
    if binary_class_names[pred_label] == "1_food":

      image_transformed = effnetb2_transform(image).unsqueeze(0).to(device) # Adding a batch dimension

      effnetb2.eval()

      with torch.inference_mode():
        pred_logits = effnetb2(image_transformed)

        pred_probs = torch.softmax(pred_logits, dim=1)

        pred_label = torch.argmax(pred_probs, dim=1)

        # Dictionary holds the class names and its given probability by the model
      pred_dict = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

      end = time.time()
      # Time it took to make the prediction
      time_to_run = round(end - start, 4)


      return pred_dict, time_to_run



    else:
      return "Image unrecognizable", "Image unrecognizable"



Test out the function

In [ ]:
import random
from PIL import Image
## Get a random image
image_paths = list(Path(test_dir).glob("*/*.jpg"))

image_path = random.sample(image_paths, k=1)[0]
print(image_path)

img = Image.open(image_path)

pred_dict, time_to_run = image_classification(img)

pred_dict, time_to_run

data/large_pizza_hamburger_lasagna_sushi_steak/test/hamburger/3305010.jpg
tensor([[1]], device='cuda:0')


({'hamburger': 0.9968607425689697,
  'lasagna': 0.00023155499366112053,
  'pizza': 0.000149762854562141,
  'steak': 0.0011195661500096321,
  'sushi': 0.001638407469727099},
 1.5374)

Get examples

In [ ]:
examples = random.sample(image_paths, k=3)
examples

[PosixPath('data/large_pizza_hamburger_lasagna_sushi_steak/test/steak/1956040.jpg'),
 PosixPath('data/large_pizza_hamburger_lasagna_sushi_steak/test/pizza/849291.jpg'),
 PosixPath('data/large_pizza_hamburger_lasagna_sushi_steak/test/sushi/72926.jpg')]

## Buid the interface

In [ ]:
demo = gr.Interface(fn=image_classification,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=5, label="Predictions"),
                             gr.Label(label="Prediction Time (s)")],
                    examples=examples,
                    title="BiteID",
                    description="Upload an image, and the computer vision model will predict whether it’s pizza, hamburger, lasagna, sushi, or steak. Try it out and see which dish it classifies!")

# Launch the demo!
demo.launch(debug=False, # print errors locally?
            share=True) # generate a publically shareable URL?

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://820cb5790fb3711df0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Creating Gradio app structure

### Creating demos folder

In [ ]:
import shutil
from pathlib import Path

biteid_demo_path = Path("demos/biteid/")

if biteid_demo_path.exists():
  shutil.rmtree(biteid_demo_path)

biteid_demo_path.mkdir(parents=True,
                       exist_ok=True)



### Creating example images folder

In [ ]:
demo_examples_path = biteidv2_demo_path / "examples"
demo_examples_path.mkdir(parents=True, exist_ok=True)

for example in examples:
  shutil.copy(example, demo_examples_path)


### Moving models to demo folder

In [ ]:

try:
  shutil.move("effnetb2_99_percent_accuracy_binary_classification.pth", "demos/biteid")
  shutil.move("large_dataset_effnetb2_90_percent_accuracy_10_epochs.pth", "demos/biteid")
except:
  print(f"Models have already been moved or they haven't been uploaded")

'demos/biteid/large_dataset_effnetb2_90_percent_accuracy_10_epochs.pth'

### Creating model.py

In [ ]:
%%writefile demos/biteid/model.py

def create_effnetb2_model(num_classes: int,
                          seed:int=42):


  # Get the weights, transforms of the model and make the model
  efficientnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  effnetb2_transform = efficientnetb2_weights.transforms()
  effnetb2 = torchvision.models.efficientnet_b2(weights=efficientnetb2_weights)

  # Freeze the base layers of efficientnetb2
  for param in effnetb2.parameters():
    param.requires_grad = False



  # Change classifier head of effnetb2 to suit 5 classes
  torch.manual_seed(seed)
  effnetb2.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408, out_features=num_classes),
  )

  return effnetb2, effnetb2_transform

Writing demos/biteid/model.py


### Creating app.py

In [ ]:
%%writefile demos/biteid/app.py

from model import create_effnetb2_model

import time
import gradio as gr
import torch

class_names = ["hamburger", "lasagna", "pizza", "steak", "sushi"]

binary_class_names = ["0_non-food", "1_food"]

effnetb2, effnetb2_transform = create_effnetb2_model(num_classes=5)

effnetb2_binary, effnetb2_transform = create_effnetb2_model(num_classes=1)


# Load state dicts
effnetb2.load_state_dict(
    torch.load(f="large_dataset_effnetb2_90_percent_accuracy_10_epochs.pth",
               map_location=torch.device("cpu")
               )
    )


effnetb2_binary.load_state_dict(
    torch.load(
        f="effnetb2_99_percent_accuracy_binary_classification.pth",
        map_location=torch.device("cpu")
    )
)



def image_classification(image):


  start = time.time()

  # First, pass image through binary classifier to see if the image is food
  image_transformed = effnetb2_transform(image).unsqueeze(dim=0).to(device)

  effnetb2_binary.eval()

  with torch.inference_mode():
    pred_logit = effnetb2_binary(image_transformed)

    pred_prob = torch.sigmoid(pred_logit)
    pred_label = (pred_prob >= 0.5).long()
    print(pred_label)
    # if the image is food:
    if binary_class_names[pred_label] == "1_food":

      image_transformed = effnetb2_transform(image).unsqueeze(0).to(device) # Adding a batch dimension

      effnetb2.eval()

      with torch.inference_mode():
        pred_logits = effnetb2(image_transformed)

        pred_probs = torch.softmax(pred_logits, dim=1)

        pred_label = torch.argmax(pred_probs, dim=1)

        # Dictionary holds the class names and its given probability by the model
      pred_dict = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

      end = time.time()
      # Time it took to make the prediction
      time_to_run = round(end - start, 4)


      return pred_dict, time_to_run

    else:
      return "Image unrecognizable", "Image unrecognizable"


examples = [["examples/" + example] for example in os.listdir("examples")]


demo = gr.Interface(fn=image_classification,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=5, label="Predictions"),
                             gr.Label(label="Prediction Time (s)")],
                    examples=examples,
                    title="BiteID🍔",
                    description="Upload an image, and the computer vision model will predict whether it’s pizza, hamburger, lasagna, sushi, or steak. Try it out and see which dish it classifies!")

# Launch the demo!
demo.launch()


Overwriting demos/biteid/app.py


### Creating requirements.txt

In [ ]:
%%writefile demos/biteid/requirements.txt
torch==2.5.1
gradio==5.9.1
torchvision==0.20.1

Writing demos/biteid/requirements.txt


In [ ]:
!python --version

Python 3.10.12


In [ ]:
!ls demos/biteid

app.py
effnetb2_99_percent_accuracy_binary_classification.pth
examples
large_dataset_effnetb2_90_percent_accuracy_10_epochs.pth
model.py
requirements.txt


### Downloading app files

In [ ]:
# Change into and zip the biteid folder. Everything after -x is excluded from being zipped
!cd demos/biteid && zip -r ../biteid.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

from google.colab import files
files.download("demos/biteid.zip")


  adding: app.py (deflated 60%)
  adding: effnetb2_99_percent_accuracy_binary_classification.pth (deflated 8%)
  adding: examples/ (stored 0%)
  adding: examples/849291.jpg (deflated 1%)
  adding: examples/72926.jpg (deflated 0%)
  adding: examples/1956040.jpg (deflated 16%)
  adding: large_dataset_effnetb2_90_percent_accuracy_10_epochs.pth (deflated 8%)
  adding: model.py (deflated 51%)
  adding: requirements.txt (deflated 4%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>